# 한글 자연어 처리: 텍스트 벡터화 (Text Vectorization)

## 개요
이 노트북에서는 KoNLPy와 TensorFlow의 TextVectorization을 결합하여 한글 텍스트를 딥러닝 모델에서 사용할 수 있는 숫자 형태로 변환하는 과정을 학습합니다.

### 주요 학습 내용:
1. 텍스트 전처리 및 정제
2. 한글 형태소 분석과 토큰화
3. TensorFlow TextVectorization 활용
4. 어휘 사전(Vocabulary) 구축
5. 텍스트의 숫자 벡터 변환

## 1. 라이브러리 및 환경 설정

In [9]:
# 필요한 라이브러리 import
from konlpy.tag import Okt
import tensorflow as tf
import re
from tensorflow.keras.layers import TextVectorization
import numpy as np

print(f"TensorFlow 버전: {tf.__version__}")

TensorFlow 버전: 2.15.1


## 2. 샘플 데이터 준비

자연어 처리 학습을 위한 한글 텍스트 데이터를 준비합니다.

In [10]:
# 학습용 텍스트 데이터 준비
sample_texts = [
    "나는 AI 챗봇을 사용합니다.",
    "자연어 처리는 어렵습니다",
    "챗봇과 대화하는것이 즐겁습니다",
    "AI로 인해 사람들이 일자리가 사라지고 있습니다."
]

print("=== 원본 텍스트 데이터 ===")
for i, text in enumerate(sample_texts, 1):
    print(f"{i}. {text}")

=== 원본 텍스트 데이터 ===
1. 나는 AI 챗봇을 사용합니다.
2. 자연어 처리는 어렵습니다
3. 챗봇과 대화하는것이 즐겁습니다
4. AI로 인해 사람들이 일자리가 사라지고 있습니다.


## 3. 텍스트 전처리 함수 정의

### 3.1 기본 텍스트 정제 함수

In [11]:
def clean_text(text):
    """
    텍스트 정제 함수
    - 대문자를 소문자로 변환
    - 한글, 영문, 숫자, 공백만 유지하고 나머지 특수문자 제거
    
    Args:
        text (str): 원본 텍스트
    
    Returns:
        str: 정제된 텍스트
    """
    # 소문자 변환
    text = text.lower()
    
    # 정규식을 사용하여 필요한 문자만 유지
    # [^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\s]: 한글, 영문, 숫자, 공백 외 모든 문자 제거
    text = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\s]", "", text)
    
    return text

# 텍스트 정제 결과 확인
print("=== 텍스트 정제 결과 ===")
for i, text in enumerate(sample_texts, 1):
    cleaned = clean_text(text)
    print(f"{i}. 원본: {text}")
    print(f"   정제: {cleaned}")
    print()

=== 텍스트 정제 결과 ===
1. 원본: 나는 AI 챗봇을 사용합니다.
   정제: 나는 ai 챗봇을 사용합니다

2. 원본: 자연어 처리는 어렵습니다
   정제: 자연어 처리는 어렵습니다

3. 원본: 챗봇과 대화하는것이 즐겁습니다
   정제: 챗봇과 대화하는것이 즐겁습니다

4. 원본: AI로 인해 사람들이 일자리가 사라지고 있습니다.
   정제: ai로 인해 사람들이 일자리가 사라지고 있습니다



## 4. 한글 형태소 분석 및 토큰화

### 4.1 형태소 분석기 초기화

In [12]:
# KoNLPy Okt 형태소 분석기 초기화
okt = Okt()

print("=== 형태소 분석 결과 ===")
for i, text in enumerate(sample_texts, 1):
    cleaned = clean_text(text)
    morphs = okt.morphs(cleaned)
    print(f"{i}. 텍스트: {cleaned}")
    print(f"   형태소: {morphs}")
    print()

=== 형태소 분석 결과 ===
1. 텍스트: 나는 ai 챗봇을 사용합니다
   형태소: ['나', '는', 'ai', '챗봇', '을', '사용', '합니다']

2. 텍스트: 자연어 처리는 어렵습니다
   형태소: ['자연어', '처리', '는', '어렵습니다']

3. 텍스트: 챗봇과 대화하는것이 즐겁습니다
   형태소: ['챗봇', '과', '대화', '하는것이', '즐겁습니다']

4. 텍스트: ai로 인해 사람들이 일자리가 사라지고 있습니다
   형태소: ['ai', '로', '인해', '사람', '들', '이', '일자리', '가', '사라지고', '있습니다']



### 4.2 커스텀 표준화 함수 정의

TextVectorization에서 사용할 전처리 함수를 정의합니다.

In [13]:
def custom_standardization_fn(text):
    """
    TextVectorization용 커스텀 표준화 함수
    1. 텍스트 정제
    2. 형태소 분석
    3. 토큰들을 공백으로 구분하여 재결합
    
    Args:
        text (str): 원본 텍스트
    
    Returns:
        str: 형태소 분석된 토큰들이 공백으로 구분된 문자열
    """
    # 1단계: 텍스트 정제
    cleaned_text = clean_text(text)
    
    # 2단계: 형태소 분석하여 토큰화
    tokens = okt.morphs(cleaned_text)
    
    # 3단계: 토큰들을 공백으로 구분하여 재결합
    return " ".join(tokens)

# 커스텀 표준화 함수 테스트
print("=== 커스텀 표준화 함수 결과 ===")
for i, text in enumerate(sample_texts, 1):
    standardized = custom_standardization_fn(text)
    print(f"{i}. 원본: {text}")
    print(f"   표준화: {standardized}")
    print()

=== 커스텀 표준화 함수 결과 ===
1. 원본: 나는 AI 챗봇을 사용합니다.
   표준화: 나 는 ai 챗봇 을 사용 합니다

2. 원본: 자연어 처리는 어렵습니다
   표준화: 자연어 처리 는 어렵습니다

3. 원본: 챗봇과 대화하는것이 즐겁습니다
   표준화: 챗봇 과 대화 하는것이 즐겁습니다

4. 원본: AI로 인해 사람들이 일자리가 사라지고 있습니다.
   표준화: ai 로 인해 사람 들 이 일자리 가 사라지고 있습니다



## 5. TextVectorization 설정 및 어휘 사전 구축

### 5.1 TextVectorization 객체 생성

In [14]:
# TextVectorization 설정 변수
MAX_TOKENS = 1000
OUTPUT_MODE = "int"
OUTPUT_SEQUENCE_LENGTH = 20

# TextVectorization 객체 생성
vectorizer = TextVectorization(
    max_tokens=MAX_TOKENS,                # 최대 어휘 사전 크기 (상위 1000개 빈도 단어)
    output_mode=OUTPUT_MODE,              # 출력 형태: 정수 시퀀스
    output_sequence_length=OUTPUT_SEQUENCE_LENGTH,  # 출력 시퀀스 최대 길이 (패딩/자르기)
    standardize=None                      # 커스텀 표준화 함수를 외부에서 적용하므로 None
)

print("TextVectorization 설정:")
print(f"- 최대 토큰 수: {MAX_TOKENS}")
print(f"- 출력 모드: {OUTPUT_MODE}")
print(f"- 시퀀스 길이: {OUTPUT_SEQUENCE_LENGTH}")

TextVectorization 설정:
- 최대 토큰 수: 1000
- 출력 모드: int
- 시퀀스 길이: 20


### 5.2 어휘 사전 학습 (Adaptation)

사전에 정의된 텍스트 데이터로부터 어휘 사전을 구축합니다.

In [16]:
# 표준화된 데이터 생성
processed_data = [custom_standardization_fn(text) for text in sample_texts]

print("=== 어휘 사전 학습용 데이터 ===")
for i, data in enumerate(processed_data, 1):
    print(f"{i}. {data}")

print("\n어휘 사전 학습 중...")

# 어휘 사전 학습
vectorizer.adapt(processed_data)

print("어휘 사전 학습 완료!")

=== 어휘 사전 학습용 데이터 ===
1. 나 는 ai 챗봇 을 사용 합니다
2. 자연어 처리 는 어렵습니다
3. 챗봇 과 대화 하는것이 즐겁습니다
4. ai 로 인해 사람 들 이 일자리 가 사라지고 있습니다

어휘 사전 학습 중...
어휘 사전 학습 완료!


## 6. 어휘 사전 분석

### 6.1 생성된 어휘 사전 확인

In [17]:
# 어휘 사전 가져오기
vocabulary = vectorizer.get_vocabulary()

print("=== 생성된 어휘 사전 ===")
print(f"총 어휘 수: {len(vocabulary)}")
print("\n어휘 목록 (인덱스 순):")
for i, word in enumerate(vocabulary[:20]):  # 상위 20개만 출력
    print(f"{i:2d}: '{word}'")

if len(vocabulary) > 20:
    print(f"... (총 {len(vocabulary)}개 어휘)")

=== 생성된 어휘 사전 ===
총 어휘 수: 25

어휘 목록 (인덱스 순):
 0: ''
 1: '[UNK]'
 2: '챗봇'
 3: '는'
 4: 'ai'
 5: '합니다'
 6: '하는것이'
 7: '처리'
 8: '즐겁습니다'
 9: '자연어'
10: '있습니다'
11: '일자리'
12: '인해'
13: '이'
14: '을'
15: '어렵습니다'
16: '사용'
17: '사람'
18: '사라지고'
19: '로'
... (총 25개 어휘)


### 6.2 단어-인덱스 매핑 딕셔너리 생성

In [18]:
# 단어를 키로, 인덱스를 값으로 하는 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(vocabulary)}

print("=== 단어-인덱스 매핑 딕셔너리 ===")
print("형식: {단어: 인덱스}")
print()

# 빈 문자열과 [UNK] 토큰 확인
special_tokens = ['', '[UNK]']
print("특수 토큰:")
for token in special_tokens:
    if token in word_to_index:
        print(f"  '{token}': {word_to_index[token]}")

print("\n일반 어휘:")
regular_words = [word for word in vocabulary if word not in special_tokens]
for word in regular_words[:10]:  # 상위 10개만 출력
    print(f"  '{word}': {word_to_index[word]}")

if len(regular_words) > 10:
    print(f"  ... (총 {len(regular_words)}개 일반 어휘)")

=== 단어-인덱스 매핑 딕셔너리 ===
형식: {단어: 인덱스}

특수 토큰:
  '': 0
  '[UNK]': 1

일반 어휘:
  '챗봇': 2
  '는': 3
  'ai': 4
  '합니다': 5
  '하는것이': 6
  '처리': 7
  '즐겁습니다': 8
  '자연어': 9
  '있습니다': 10
  '일자리': 11
  ... (총 23개 일반 어휘)


## 7. 텍스트 벡터화 실험

### 7.1 새로운 텍스트 벡터화 테스트

In [19]:
# 테스트용 새로운 텍스트들
test_texts = [
    "챗봇과 이야기를 합니다",
    "챗봇보다 재미나이가 더 좋아요", 
    "자연어 처리는 어려워요"
]

print("=== 테스트 텍스트 벡터화 ===")
print()

# 각 테스트 텍스트에 대해 단계별 처리 과정 보여주기
for i, text in enumerate(test_texts, 1):
    print(f"테스트 {i}: {text}")
    
    # 1단계: 표준화
    standardized = custom_standardization_fn(text)
    print(f"  표준화: {standardized}")
    
    # 2단계: 벡터화
    vectorized = vectorizer([standardized])
    print(f"  벡터화: {vectorized.numpy()[0]}")
    
    # 3단계: 벡터를 단어로 역변환하여 확인
    tokens = standardized.split()
    token_indices = []
    for token in tokens:
        if token in word_to_index:
            token_indices.append(word_to_index[token])
        else:
            token_indices.append(word_to_index['[UNK]'] if '[UNK]' in word_to_index else 1)
    
    print(f"  토큰별 인덱스: {dict(zip(tokens, token_indices))}")
    print()

=== 테스트 텍스트 벡터화 ===

테스트 1: 챗봇과 이야기를 합니다
  표준화: 챗봇 과 이야기 를 합니다
  벡터화: [ 2 23  1  1  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
  토큰별 인덱스: {'챗봇': 2, '과': 23, '이야기': 1, '를': 1, '합니다': 5}

테스트 2: 챗봇보다 재미나이가 더 좋아요
  표준화: 챗봇 보다 재미나 이 가 더 좋아요
  벡터화: [ 2  1  1 13 24  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
  토큰별 인덱스: {'챗봇': 2, '보다': 1, '재미나': 1, '이': 13, '가': 24, '더': 1, '좋아요': 1}

테스트 3: 자연어 처리는 어려워요
  표준화: 자연어 처리 는 어려워요
  벡터화: [9 7 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  토큰별 인덱스: {'자연어': 9, '처리': 7, '는': 3, '어려워요': 1}



### 7.2 배치 벡터화 테스트

In [20]:
# 모든 테스트 텍스트를 한 번에 벡터화
standardized_test_texts = [custom_standardization_fn(text) for text in test_texts]
batch_vectorized = vectorizer(standardized_test_texts)

print("=== 배치 벡터화 결과 ===")
print(f"벡터화된 텐서 형태: {batch_vectorized.shape}")
print(f"(배치 크기: {batch_vectorized.shape[0]}, 시퀀스 길이: {batch_vectorized.shape[1]})")
print()

print("각 텍스트별 벡터:")
for i, (original, vector) in enumerate(zip(test_texts, batch_vectorized.numpy())):
    print(f"{i+1}. '{original}'")
    print(f"   벡터: {vector}")
    # 0이 아닌 값들만 출력 (패딩 제거)
    non_zero = vector[vector != 0]
    print(f"   유효값: {non_zero}")
    print()

=== 배치 벡터화 결과 ===
벡터화된 텐서 형태: (3, 20)
(배치 크기: 3, 시퀀스 길이: 20)

각 텍스트별 벡터:
1. '챗봇과 이야기를 합니다'
   벡터: [ 2 23  1  1  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
   유효값: [ 2 23  1  1  5]

2. '챗봇보다 재미나이가 더 좋아요'
   벡터: [ 2  1  1 13 24  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
   유효값: [ 2  1  1 13 24  1  1]

3. '자연어 처리는 어려워요'
   벡터: [9 7 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
   유효값: [9 7 3 1]



## 8. 결과 분석 및 시각화

### 8.1 어휘 빈도 분석

In [21]:
# 전체 데이터에서 토큰 빈도 계산
from collections import Counter

all_tokens = []
for text in sample_texts:
    tokens = custom_standardization_fn(text).split()
    all_tokens.extend(tokens)

token_freq = Counter(all_tokens)

print("=== 토큰 빈도 분석 ===")
print(f"총 토큰 수: {len(all_tokens)}")
print(f"고유 토큰 수: {len(token_freq)}")
print()

print("빈도별 토큰 순위:")
for i, (token, freq) in enumerate(token_freq.most_common(10), 1):
    index = word_to_index.get(token, 'Unknown')
    print(f"{i:2d}. '{token}': {freq}회 (인덱스: {index})")

=== 토큰 빈도 분석 ===
총 토큰 수: 26
고유 토큰 수: 23

빈도별 토큰 순위:
 1. '는': 2회 (인덱스: 3)
 2. 'ai': 2회 (인덱스: 4)
 3. '챗봇': 2회 (인덱스: 2)
 4. '나': 1회 (인덱스: 22)
 5. '을': 1회 (인덱스: 14)
 6. '사용': 1회 (인덱스: 16)
 7. '합니다': 1회 (인덱스: 5)
 8. '자연어': 1회 (인덱스: 9)
 9. '처리': 1회 (인덱스: 7)
10. '어렵습니다': 1회 (인덱스: 15)


### 8.2 벡터화 과정 요약

텍스트를 딥러닝 모델에서 사용할 수 있는 숫자 벡터로 변환하는 전체 과정을 정리하면 다음과 같습니다:

#### 1단계: 텍스트 전처리 (clean_text)
- 소문자 변환
- 특수문자 제거 (한글, 영문, 숫자, 공백만 유지)

#### 2단계: 형태소 분석 (KoNLPy Okt)
- 의미 단위로 토큰 분리
- 공백으로 토큰 재결합

#### 3단계: 어휘 사전 구축 (TextVectorization.adapt)
- 빈도 기반 어휘 사전 생성
- 각 토큰에 고유 인덱스 할당

#### 4단계: 벡터화 (TextVectorization.__call__)
- 토큰을 인덱스로 변환
- 시퀀스 길이 통일 (패딩/자르기)

**최종 결과**: 텍스트 → 길이 20의 정수 벡터

이 과정을 통해 자연어 텍스트를 딥러닝 모델이 이해할 수 있는 수치 데이터로 변환할 수 있습니다.

## 9. 실전 활용 예시

### 9.1 새로운 텍스트 처리 함수

In [22]:
def process_new_text(text, show_steps=True):
    """
    새로운 텍스트를 처리하여 모델 입력용 벡터로 변환
    
    Args:
        text (str): 처리할 텍스트
        show_steps (bool): 중간 과정 출력 여부
    
    Returns:
        numpy.ndarray: 벡터화된 결과
    """
    if show_steps:
        print(f"입력 텍스트: '{text}'")
    
    # 표준화
    standardized = custom_standardization_fn(text)
    if show_steps:
        print(f"표준화 결과: '{standardized}'")
    
    # 벡터화
    vector = vectorizer([standardized]).numpy()[0]
    if show_steps:
        print(f"벡터화 결과: {vector}")
        non_zero_indices = np.nonzero(vector)[0]
        if len(non_zero_indices) > 0:
            print(f"유효 토큰 수: {len(non_zero_indices)}")
        print()
    
    return vector

# 실전 예시
example_texts = [
    "AI 기술이 발전하고 있어요",
    "머신러닝은 정말 흥미로운 분야입니다",
    "코딩하는 것이 즐거워요"
]

print("=== 실전 텍스트 처리 예시 ===")
processed_vectors = []
for text in example_texts:
    vector = process_new_text(text)
    processed_vectors.append(vector)

# 처리된 벡터들을 배열로 변환 (모델 입력용)
input_array = np.array(processed_vectors)
print(f"최종 입력 배열 형태: {input_array.shape}")
print("이 배열을 딥러닝 모델의 입력으로 사용할 수 있습니다.")

=== 실전 텍스트 처리 예시 ===
입력 텍스트: 'AI 기술이 발전하고 있어요'
표준화 결과: 'ai 기술 이 발전 하고 있어요'
벡터화 결과: [ 4  1 13  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
유효 토큰 수: 6

입력 텍스트: '머신러닝은 정말 흥미로운 분야입니다'
표준화 결과: '머신 러닝 은 정말 흥미로운 분야 입니다'
벡터화 결과: [1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
유효 토큰 수: 7

입력 텍스트: '코딩하는 것이 즐거워요'
표준화 결과: '코딩 하는 것 이 즐거워요'
벡터화 결과: [ 1  1  1 13  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
유효 토큰 수: 5

최종 입력 배열 형태: (3, 20)
이 배열을 딥러닝 모델의 입력으로 사용할 수 있습니다.
